# **Taller 4: Optimización de Campañas de Marketing y Variables Clave**

### **Contexto del Caso**

Un banco portugués lo ha contratado como consultor de ciencia de datos. El banco tiene un problema de eficiencia: sus campañas de telemercadeo para ofrecer depósitos a plazo tienen una tasa de éxito muy baja. Se invierte mucho tiempo y recursos (costos de call center) llamando a clientes que no están interesados.

**Su misión:** Construir y optimizar modelos de Machine Learning que predigan qué clientes tienen mayor probabilidad de decir **"sí"** a la oferta (`y = 'yes'`). 

**Entregable Adicional:** El banco no solo quiere un modelo preciso, también quiere entender **POR QUÉ** un cliente es un buen prospecto. Su segundo objetivo es identificar cuáles son las **variables más relevantes** que usan los modelos para tomar sus decisiones. Esto permitirá al banco no solo enfocar sus llamadas, sino también crear mejores guiones de marketing y entender mejor a su clientela.

**El Dataset:** `bank-additional-full.csv`. Contiene información de más de 40,000 contactos de telemercadeo, incluyendo datos demográficos del cliente, información socioeconómica (tasa de interés, índice de precios al consumidor) e información de la campaña (último contacto, resultado anterior).

**Temas a Cubrir:**
1.  Preprocesamiento de datos (Pipelines, ColumnTransformer).
2.  `KNeighborsClassifier` + `GridSearchCV`.
3.  `DecisionTreeClassifier` + `GridSearchCV`.
4.  Análisis de Importancia de Variables (Feature Importance).

---

## 1. Preparación del Entorno y Datos

### 1.1. Carga de Librerías

Importamos todas las herramientas que necesitaremos.

In [ ]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Modelos y herramientas de Scikit-Learn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.inspection import permutation_importance # <-- ¡Importante para KNN!

# Configuraciones para una mejor visualización
plt.style.use('seaborn-v0_8-whitegrid')
import warnings
warnings.filterwarnings('ignore')

### 1.2. Carga y Exploración Inicial

In [ ]:
# 1.2.1: Cargue el dataset desde la siguiente URL.
# ¡Cuidado! Este archivo usa punto y coma (;) como separador.
url = 'https://raw.githubusercontent.com/Fod-Sol/m-carter-MDS-de/main/Data/bank-additional-full.csv'

# ### TU CÓDIGO AQUÍ ###
# Usa pd.read_csv() con el argumento sep=';'
df = pd.read_csv(url, sep=';')

# 1.2.2: Muestre las primeras 5 filas
# ### TU CÓDIGO AQUÍ ###
display(df.head())

In [ ]:
# 1.2.3: Use .info() para revisar los tipos de datos y los nulos
# ### TU CÓDIGO AQUÍ ###
df.info()

**Análisis de `info()`:**
* Tenemos 41188 registros.
* ¡No hay valores nulos! (Esto es raro y una buena noticia).
* Tenemos muchas columnas `object` (categóricas) que deberemos transformar: `job`, `marital`, `education`, etc.
* Tenemos varias columnas numéricas (float e int) que deberemos escalar: `age`, `duration`, `campaign`, `euribor3m`, etc.

In [ ]:
# 1.2.4: Revise el balance de la variable objetivo 'y'
# Use .value_counts() con normalize=True

# ### TU CÓDIGO AQUÍ ###
print("Distribución de la variable objetivo 'y':")
print(df['y'].value_counts(normalize=True))

**Análisis del Target:**
El dataset está **muy desbalanceado**. Casi el 89% de los clientes dijeron 'no' y solo el 11% dijo 'yes'.
**Implicación Económica:** Esto es normal. Las campañas de marketing tienen bajas tasas de conversión. ¡Por eso es tan importante optimizar! 
**Implicación Técnica:** El `accuracy` no será una buena métrica. Un modelo que siempre diga 'no' tendrá 89% de accuracy. Deberemos enfocarnos en las métricas de la clase 'yes' (como `recall` y `precision`) dentro del `classification_report`.

---

## 2. Preprocesamiento (Usando Pipelines)

Vamos a definir nuestro `X` e `y`, y luego crear un `ColumnTransformer` que se encargue de aplicar `StandardScaler` a los números y `OneHotEncoder` a las categorías.

In [ ]:
# 2.1: Separar X (predictoras) e y (objetivo)

# ### TU CÓDIGO AQUÍ ###
X = df.drop('y', axis=1)
y = df['y']

# 2.2: Dividir en train y test (80/20)
# ¡Use stratify=y para mantener la proporción de 'yes' y 'no' en ambos sets!

# ### TU CÓDIGO AQUÍ ###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamaño Train: {X_train.shape}")
print(f"Tamaño Test: {X_test.shape}")

In [ ]:
# 2.3: Identificar automáticamente las columnas numéricas y categóricas

# ### TU CÓDIGO AQUÍ ###
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

print("Columnas Numéricas:")
print(numerical_features)
print("\nColumnas Categóricas:")
print(categorical_features)

In [ ]:
# 2.4: Crear el ColumnTransformer (preprocessor)

# Crear los transformadores individuales
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))]) # handle_unknown='ignore' es clave

# Unirlos en el preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

---

## 3. Modelo 1: KNN y la Búsqueda del `k` Óptimo

Ahora uniremos el preprocesador y el clasificador KNN en un solo `Pipeline` y usaremos `GridSearchCV` para encontrar el mejor `k`.

In [ ]:
# 3.1: Crear el Pipeline completo de KNN
# (une el 'preprocessor' con el modelo 'KNeighborsClassifier')

# ### TU CÓDIGO AQUÍ ###
knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsClassifier())
])

# 3.2: Definir la grilla de hiperparámetros para KNN
# Queremos probar k = 3, 5, 7, 11 (valores impares para evitar empates)
# Pista: El nombre DEBE ser 'model__n_neighbors' (por el nombre en el pipeline)

# ### TU CÓDIGO AQUÍ ###
param_grid_knn = {
    'model__n_neighbors': [3, 5, 7, 11]
}

# 3.3: Configurar y ejecutar GridSearchCV
# Use 3 folds (cv=3) para que corra más rápido.
# Use scoring='accuracy' por ahora, aunque sabemos que es desbalanceado.

# ### TU CÓDIGO AQUÍ ###
grid_knn = GridSearchCV(knn_pipeline, param_grid_knn, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)

print("Iniciando GridSearchCV para KNN...")
grid_knn.fit(X_train, y_train)
print("GridSearchCV para KNN completado.")

# 3.4: Mostrar los mejores resultados
# ### TU CÓDIGO AQUÍ ###
print(f"Mejor valor de 'k' para KNN: {grid_knn.best_params_}")
print(f"Mejor Accuracy (CV): {grid_knn.best_score_:.4f}")

---

## 4. Modelo 2: Árbol de Decisión y la Búsqueda del "Punto Óptimo"

Repetiremos el proceso con un Árbol de Decisión. Esta vez, las "perillas" (hiperparámetros) que ajustaremos serán `max_depth` (para evitar sobreajuste) y `min_samples_leaf`.

In [ ]:
# 4.1: Crear el Pipeline completo para el Árbol de Decisión
# (une el 'preprocessor' con el modelo 'DecisionTreeClassifier')

# ### TU CÓDIGO AQUÍ ###
tree_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DecisionTreeClassifier(random_state=42))
])

# 4.2: Definir la grilla de hiperparámetros para el Árbol
# Probaremos 'max_depth' = [3, 5, 7]
# Y 'min_samples_leaf' = [20, 50, 100] (para controlar la complejidad)

# ### TU CÓDIGO AQUÍ ###
param_grid_tree = {
    'model__max_depth': [3, 5, 7],
    'model__min_samples_leaf': [20, 50, 100]
}

# 4.3: Configurar y ejecutar GridSearchCV
# ### TU CÓDIGO AQUÍ ###
grid_tree = GridSearchCV(tree_pipeline, param_grid_tree, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)

print("Iniciando GridSearchCV para Árbol de Decisión...")
grid_tree.fit(X_train, y_train)
print("GridSearchCV para Árbol de Decisión completado.")

# 4.4: Mostrar los mejores resultados
# ### TU CÓDIGO AQUÍ ###
print(f"Mejores hiperparámetros para el Árbol: {grid_tree.best_params_}")
print(f"Mejor Accuracy (CV): {grid_tree.best_score_:.4f}")

---

## 5. Evaluación Final y Recomendación de Modelo

El Árbol de Decisión probablemente dio un mejor `accuracy` (y es más rápido e interpretable). Vamos a declararlo nuestro **modelo ganador** y evaluarlo en el `test set` (nuestro examen final imparcial).

In [ ]:
# 5.1: Obtener el mejor modelo de árbol (el 'best_estimator_')

# ### TU CÓDIGO AQUÍ ###
best_tree_model = grid_tree.best_estimator_

# 5.2: Realizar predicciones sobre el conjunto de PRUEBA (X_test)

# ### TU CÓDIGO AQUÍ ###
y_pred_tree = best_tree_model.predict(X_test)

# 5.3: Imprimir el Reporte de Clasificación
print("--- Reporte de Clasificación Final (Árbol Optimizado) ---")

# ### TU CÓDIGO AQUÍ ###
print(classification_report(y_test, y_pred_tree))

**Análisis del Reporte:**
* Observe el `accuracy` general (probablemente ~90%).
* Ahora mire la fila de `'yes'`: ¿Cuál es el `precision`? ¿Cuál es el `recall`?
* **Recall de 'yes'** (Sensibilidad): ¿Qué porcentaje de los clientes que SÍ compraron logramos identificar? (Usualmente lo más importante para el banco, para no perder oportunidades).
* **Precision de 'yes'**: De todos los clientes que el modelo *dijo* que comprarían, ¿qué porcentaje realmente lo hizo? (Importante para no gastar llamadas).

In [ ]:
# 5.4: Visualizar la Matriz de Confusión
cm = confusion_matrix(y_test, y_pred_tree, labels=best_tree_model.classes_)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=best_tree_model.classes_,
            yticklabels=best_tree_model.classes_)
plt.ylabel('Verdadero')
plt.xlabel('Predicho')
plt.title('Matriz de Confusión - Árbol Optimizado')
plt.show()

---

## 6. ¿Cuáles son las Variables Más Relevantes?

Esta es la segunda parte de la solicitud del banco. Necesitamos explicar *por qué* el modelo toma sus decisiones.

### 6.1. Importancia de Variables (Árbol de Decisión)

Los árboles de decisión calculan esto automáticamente. Miden cuánto reduce la impureza (Gini) cada variable en promedio.

In [ ]:
# 6.1.1: Extraer el modelo de árbol y el preprocesador del pipeline optimizado
# (Ya tenemos 'best_tree_model')

# ### TU CÓDIGO AQUÍ ###
final_tree_model = best_tree_model.named_steps['model']
final_preprocessor = best_tree_model.named_steps['preprocessor']

# 6.1.2: Obtener los nombres de las características DESPUÉS del OneHotEncoding

# ### TU CÓDIGO AQUÍ ###
encoded_feature_names = final_preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
all_feature_names = list(numerical_features) + list(encoded_feature_names)

# 6.1.3: Obtener las importancias (del 'final_tree_model')

# ### TU CÓDIGO AQUÍ ###
importances = final_tree_model.feature_importances_

# 6.1.4: Crear un DataFrame para visualizarlas
tree_importance_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print("--- Top 15 Variables (Árbol de Decisión) ---")
display(tree_importance_df.head(15))

In [ ]:
# 6.1.5: Graficar las 15 variables más importantes
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=tree_importance_df.head(15))
plt.title('Top 15 Variables Más Importantes (Árbol de Decisión)')
plt.show()

**Análisis (Árbol):**
* Observe las variables que aparecen en el top. 
* `duration` (duración de la llamada) casi siempre es la #1. **Pregunta Económica:** ¿Es esta variable útil? No podemos saber la duración *antes* de hacer la llamada. Es un gran predictor, pero no es *accionable* para decidir *a quién* llamar. Es más un resultado.
* Fíjese en las siguientes: `euribor3m` (indicador económico), `poutcome_success` (si la campaña anterior fue un éxito), `pdays` (días desde el último contacto) y `age` suelen ser importantes. ¡Estas SÍ son accionables!

### 6.2. Importancia de Variables (KNN)

KNN no tiene un atributo `.feature_importances_`. Para él, usamos **Importancia por Permutación**. 

**Idea:** ¿Qué tanto empeora el modelo (cae el `accuracy`) si "barajamos" (permutamos) aleatoriamente los valores de una sola variable? Si el modelo empeora mucho, esa variable era muy importante.

In [ ]:
# 6.2.1: Obtener el mejor modelo KNN
best_knn_model = grid_knn.best_estimator_

print("Calculando Importancia por Permutación para KNN (puede tardar 1-2 minutos)...")

# 6.2.2: Ejecutar permutation_importance
# Lo corremos sobre el test set para ver qué variables importan en datos nuevos.

# ### TU CÓDIGO AQUÍ ###
perm_importance = permutation_importance(
    best_knn_model, 
    X_test, 
    y_test, 
    n_repeats=5,       # Repetir 5 veces para un resultado estable
    random_state=42,
    n_jobs=-1
)

# 6.2.3: Crear un DataFrame con los resultados
knn_importance_df = pd.DataFrame({
    'Feature': all_feature_names, # Usamos los mismos nombres de antes
    'Importance_mean': perm_importance.importances_mean # Caída promedio en accuracy
}).sort_values(by='Importance_mean', ascending=False)

print("--- Top 15 Variables (KNN por Permutación) ---")
display(knn_importance_df.head(15))

In [ ]:
# 6.2.4: Graficar las 15 variables más importantes
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance_mean', y='Feature', data=knn_importance_df.head(15))
plt.title('Top 15 Variables Más Importantes (KNN por Permutación)')
plt.xlabel('Caída promedio en Accuracy (Importancia)')
plt.show()

**Análisis (KNN):**
Compare este gráfico con el del Árbol de Decisión. 
* ¿Están de acuerdo? 
* Es probable que `duration` siga siendo la #1.
* ¿Coinciden en las variables #2, #3 y #4? 

El hecho de que dos modelos fundamentalmente diferentes (uno basado en reglas, otro en distancias) coincidan en las variables más importantes nos da **mucha confianza** para nuestra recomendación al banco.

---

## 7. Conclusión y Recomendación de Negocio

Es hora de traducir nuestros hallazgos en una recomendación de negocio clara.

In [ ]:
# 7.1: Visualicemos el árbol de decisión final para encontrar reglas
# (Usaremos max_depth=3 para que sea legible)
small_tree_model = DecisionTreeClassifier(max_depth=3, min_samples_leaf=50, random_state=42)
small_tree_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', small_tree_model)
])

small_tree_pipeline.fit(X_train, y_train)

plt.figure(figsize=(25, 12))
plot_tree(small_tree_pipeline.named_steps['model'],
          feature_names=all_feature_names,
          class_names=best_tree_model.classes_,
          filled=True,
          fontsize=10,
          rounded=True)
plt.title("Árbol de Decisión Simplificado (para Reglas de Negocio)")
plt.show()

### **7.2: Tarea de Consultoría (Su Turno)**

Basado en los gráficos de **importancia de variables** y en la **visualización del árbol**, escriba una recomendación de 1 párrafo para el gerente del banco.

**Puntos a incluir:**
1.  ¿Qué modelo recomienda usar (Árbol) y por qué (interpretable, buen rendimiento)?
2.  **Excluyendo `duration`**, ¿cuáles son las 3 variables más importantes en las que el banco debería fijarse para decidir a quién llamar?
3.  Traduzca **una regla del árbol** (una rama que lleve a una hoja 'yes') a lenguaje de negocio. (Ej: "Si el cliente tuvo éxito en la campaña anterior Y el índice euribor3m es bajo, la probabilidad de que acepte es alta.")

> **### ESCRIBA SU RECOMENDACIÓN AQUÍ ###**
> 
> ...
> 
> ...